In [1]:
%load_ext autoreload
%autoreload 2

# Datasets

## Empathetic Dialogue Data

In [4]:
import pandas as pd
import re

import torch
from transformers import WEIGHTS_NAME, AutoConfig, AutoModelForCausalLM, AutoTokenizer

#tokenizer_name = "microsoft/DialoGPT-medium"
#model_name = "microsoft/DialoGPT-medium"
tokenizer_name = "../../models/ERICA-update"
model_name ="../../models/ERICA-update"
config_name ="../../models/ERICA-update"

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name
)
model = AutoModelForCausalLM.from_pretrained(model_name, config=config_name)

In [5]:
model.config

GPT2Config {
  "_name_or_path": "../../models/ERICA-update",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 1000,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "no_repeat_ngram_size": 3,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "temperature": 0.8,
  "top_k": 100,
  "top_p": 0.7,
  "torch_dtype": "float32",
  "transformer

In [6]:
tokenizer.eos_token

'<|endoftext|>'

In [7]:
tokenizer.pad_token = "<|pad|>"

In [8]:
tokenizer.pad_token_id

50256

In [29]:
dataset_type = "valid"

data_filename = f"../../data/empatheticdialogues/{dataset_type}.csv"

In [30]:
df = pd.read_csv(data_filename, on_bad_lines="skip", delimiter=",", lineterminator="\n")

In [31]:
df.shape

(6318, 8)

In [32]:
df.iloc[112]

conv_id                                           hit:217_conv:435
utterance_idx                                                    3
context                                                    furious
prompt           I was just walking out of Taco bell. The botto...
speaker_idx                                                     60
utterance        It's not like I could back in. I waited in lin...
selfeval                                               5|5|5_5|5|5
tags                                                           NaN
Name: 112, dtype: object

Going to try:

1. do sentiment analysis on the text
2. then pipe it through the classifier and only assign a context label according to whether they are negative|neutral|positive.
 
 
example for 2:
text determined to be negative; only assign context labels that are associated with that, e.g. afraid, sad, guilty, ashamed

In [33]:
df["context"].value_counts()

surprised       336
annoyed         282
impressed       231
sad             229
excited         225
embarrassed     218
prepared        217
lonely          217
disappointed    215
confident       212
afraid          207
jealous         202
content         199
angry           196
joyful          192
terrified       190
sentimental     189
hopeful         188
proud           187
disgusted       187
anxious         183
nostalgic       179
furious         179
apprehensive    174
devastated      174
grateful        172
anticipating    171
trusting        170
caring          165
guilty          161
ashamed         159
faithful        112
Name: context, dtype: int64

In [34]:
context = sorted(list(df["context"].value_counts().keys()))

In [39]:
from copy import copy

labels = copy(context)

In [41]:
sentiment_context_map = {
    "negative": [
    ],
    "neutral": [
    ],
    "positive": [
    ],
}

context_sentiment = {}

for label in labels:
    score = input(f"{label}: ")
    context_sentiment[label] = float(score)    
    
print(context_sentiment)

afraid:  -1
angry:  -1
annoyed:  -1
anticipating:  0
anxious:  -1
apprehensive:  -1
ashamed:  -1
caring:  1
confident:  1
content:  1
devastated:  -1
disappointed:  -1
disgusted:  -1
embarrassed:  -1
excited:  1
faithful:  1
furious:  -1
grateful:  1
guilty:  -1
hopeful:  1
impressed:  1
jealous:  -1
joyful:  1
lonely:  -1
nostalgic:  1
prepared:  0
proud:  1
sad:  -1
sentimental:  0
surprised:  0
terrified:  -1
trusting:  1


{'afraid': -1.0, 'angry': -1.0, 'annoyed': -1.0, 'anticipating': 0.0, 'anxious': -1.0, 'apprehensive': -1.0, 'ashamed': -1.0, 'caring': 1.0, 'confident': 1.0, 'content': 1.0, 'devastated': -1.0, 'disappointed': -1.0, 'disgusted': -1.0, 'embarrassed': -1.0, 'excited': 1.0, 'faithful': 1.0, 'furious': -1.0, 'grateful': 1.0, 'guilty': -1.0, 'hopeful': 1.0, 'impressed': 1.0, 'jealous': -1.0, 'joyful': 1.0, 'lonely': -1.0, 'nostalgic': 1.0, 'prepared': 0.0, 'proud': 1.0, 'sad': -1.0, 'sentimental': 0.0, 'surprised': 0.0, 'terrified': -1.0, 'trusting': 1.0}


In [43]:
context_sentiment

{'afraid': -1.0,
 'angry': -1.0,
 'annoyed': -1.0,
 'anticipating': 0.0,
 'anxious': -1.0,
 'apprehensive': -1.0,
 'ashamed': -1.0,
 'caring': 1.0,
 'confident': 1.0,
 'content': 1.0,
 'devastated': -1.0,
 'disappointed': -1.0,
 'disgusted': -1.0,
 'embarrassed': -1.0,
 'excited': 1.0,
 'faithful': 1.0,
 'furious': -1.0,
 'grateful': 1.0,
 'guilty': -1.0,
 'hopeful': 1.0,
 'impressed': 1.0,
 'jealous': -1.0,
 'joyful': 1.0,
 'lonely': -1.0,
 'nostalgic': 1.0,
 'prepared': 0.0,
 'proud': 1.0,
 'sad': -1.0,
 'sentimental': 0.0,
 'surprised': 0.0,
 'terrified': -1.0,
 'trusting': 1.0}

In [42]:
import json

with open("../data/context_sentiment.json", "w+") as f:
    json.dump(context_sentiment, f)

In [38]:
df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:3_conv:6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,NaN
1,hit:3_conv:6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,NaN
2,hit:3_conv:6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,NaN
3,hit:4_conv:9,1,surprised,I was walking through my hallway a few week ag...,8,A few weeks ago_comma_ I was walking through m...,5|5|5_3|5|5,NaN
4,hit:4_conv:9,3,surprised,I was walking through my hallway a few week ag...,8,I may have let out a scream that will have him...,5|5|5_3|5|5,NaN
...,...,...,...,...,...,...,...,...
6313,hit:12350_conv:24701,5,prepared,When I went into labor with my son_comma_ it w...,791,In the end_comma_ yes--growing too big is actu...,5|5|5_5|5|5,NaN
6314,hit:12361_conv:24722,1,prepared,One time I studied all night for my final exam!,547,One time I studied all night for my final exam!,4|4|5_4|4|3,NaN
6315,hit:12361_conv:24722,3,prepared,One time I studied all night for my final exam!,547,It was for Organic Chemistry,4|4|5_4|4|3,NaN
6316,hit:12392_conv:24785,1,furious,One of my coworkers has been arguing with his ...,791,One of my coworkers has been arguing with his ...,4|5|5_5|5|5,NaN


In [17]:
# Check the lengths of each utterance
df["tokenized_len"] = df["utterance"].apply(lambda x: len(tokenizer.encode(x) + [tokenizer.eos_token_id]))

Token indices sequence length is longer than the specified maximum sequence length for this model (1842 > 1024). Running this sequence through the model will result in indexing errors


In [18]:
tokenizer.max_len_single_sentence

1024

In [19]:
model.config

GPT2Config {
  "_name_or_path": "../../models/ERICA-update",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 1000,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "no_repeat_ngram_size": 3,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "temperature": 0.8,
  "top_k": 100,
  "top_p": 0.7,
  "torch_dtype": "float32",
  "transformer

In [20]:
# Filter out anything that exceed the model max length tokenizer
exceeds_length = df[df["tokenized_len"] > tokenizer.max_len_single_sentence]
print(exceeds_length.shape)
exceeds_length["utterance"]

(4, 9)


1111    My grandpa is coming to visit!,5|5|5_5|5|5,\nh...
1113    I ate a gas station burrito for lunch. It was ...
1144    I am waiting to receive my final grade for a s...
4103    I am by myself most of the time in a isolated ...
Name: utterance, dtype: object

In [26]:
max_len = tokenizer.max_len_single_sentence
#max_len = 128
df[df["tokenized_len"] <= max_len].shape

(5695, 9)

In [27]:
# Get rows that do not exceed the length
#df = df[~df["conv_id"].isin(exceeds_length["conv_id"])].reset_index(drop=True)
df = df[df["tokenized_len"] <= max_len]

In [28]:
df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,tokenized_len
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,0,Yeah about 10 years ago I had a horrifying exp...,2|2|5_5|5|5,NaN,40
1,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,0,No I wasn't hit. It turned out they were drunk...,2|2|5_5|5|5,NaN,24
2,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,0,I don't know I was new to driving and hadn't e...,2|2|5_5|5|5,NaN,32
3,hit:34_conv:69,1,caring,My mother stopped by my house one day and said...,45,Well_comma_ can you tell me about your experie...,5|5|5_3|2|2,NaN,19
4,hit:34_conv:69,3,caring,My mother stopped by my house one day and said...,45,Oh my goodness_comma_ that's very scary! I hop...,5|5|5_3|2|2,NaN,29
...,...,...,...,...,...,...,...,...,...
5696,hit:12413_conv:24826,3,grateful,I'm glad that life is being good to me,437,Glad you think so as well!,5|5|5_4|3|4,NaN,9
5697,hit:12416_conv:24832,1,disgusted,I saw a huge cockroach outside my house today....,481,I saw a huge cockroach outside my house today!,5|5|5_4|3|4,NaN,12
5698,hit:12416_conv:24832,3,disgusted,I saw a huge cockroach outside my house today....,481,Not yet since it's the weekend. We live in Tex...,5|5|5_4|3|4,NaN,33
5699,hit:12423_conv:24847,1,anxious,I have a big test on Monday. I am so nervous_c...,481,I have a big test on Monday_comma_ I am so ner...,5|5|5_5|5|5,NaN,17


In [249]:
df.sort_values(by="tokenized_len", ascending=False)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,tokenized_len
75940,hit:12321_conv:24642,3,trusting,My mom said she'd go to the doctor the next ti...,273,Sort of.. we have a neighbor that's been by he...,4|5|5_5|5|5,NaN,128
31681,hit:5110_conv:10220,3,furious,I had to try to start my old broken lawn mower...,296,No_comma_ but I couldn't believe it because al...,5|5|5_5|5|5,NaN,126
34279,hit:5471_conv:10943,5,joyful,I watched the new Smash Bros video and they're...,303,It's like a fighting game_comma_ but some peop...,5|5|5_5|5|5,NaN,126
27047,hit:4330_conv:8660,7,sad,Life can be really not a happy place. It pains...,296,THANK YOU! That sounds GREAT! yes the raw grou...,5|5|5_5|5|5,NaN,117
67866,hit:11017_conv:22034,1,apprehensive,I got a call about a job offer back in late Ma...,384,I had been unemployed for a long time until ea...,5|5|5_4|5|5,NaN,116
...,...,...,...,...,...,...,...,...,...
41868,hit:6587_conv:13175,3,anticipating,I am waiting on to see my son talk,329,Yes,5|5|5_5|4|2,<UNIGRAM>,2
41870,hit:6587_conv:13175,5,anticipating,I am waiting on to see my son talk,329,yes,5|5|5_5|4|2,<UNIGRAM>,2
44864,hit:7014_conv:14029,1,lonely,I had to eat dinner alone today. Husband is on...,397,5,5|5|5_2|3|3,<UNIGRAM> <NUMERAL>,2
41377,hit:6515_conv:13031,4,hopeful,I hope to find a new job this week,238,cool,5|5|5_5|5|5,<UNIGRAM>,2


In [250]:
# Prepend the context (e.g. sentimental, sad, etc.) to the utterance
# df["text"] = df["context"] + " " + df["speaker_idx"].apply(lambda s: f"<speaker{str(s % 2)}>") + df["utterance"] + df["speaker_idx"].apply(lambda s: f"</speaker{str(s % 2)}>")
#df["text"] = df["context"] + " " + df["utterance"].apply(lambda x: x.replace("_comma_", ","))
# df["text"] = df["context"] + " " + df["utterance"] + df["speaker_idx"].apply(lambda s: f"<s{str(s % 2)}>")
#df.apply(lambda x: x.prompt in x.utterance, axis=1)
df["text"] = df["context"] + " " + df["utterance"]

In [251]:
df["text"][0]

'sentimental I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.'

In [252]:
print(df.iloc[0]["prompt"])
print(df.iloc[0]["utterance"])

I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.
I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.


In [253]:
df.iloc[0]["prompt"]

'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.'

In [254]:
turn_token = tokenizer.eos_token
concat_df = df.groupby(["conv_id", "context"])["text"].apply(turn_token.join).reset_index()

In [255]:
text = concat_df.iloc[0].text
text

'sentimental I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.<|endoftext|>sentimental Was this a friend you were in love with_comma_ or just a best friend?<|endoftext|>sentimental This was a best friend. I miss her.<|endoftext|>sentimental Where has she gone?<|endoftext|>sentimental We no longer talk.<|endoftext|>sentimental Oh was this something that happened because of an argument?'

In [256]:
texts = text.split(turn_token)
texts

['sentimental I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.',
 'sentimental Was this a friend you were in love with_comma_ or just a best friend?',
 'sentimental This was a best friend. I miss her.',
 'sentimental Where has she gone?',
 'sentimental We no longer talk.',
 'sentimental Oh was this something that happened because of an argument?']

In [257]:
# Get the count of utterances in each row
utterance_counts = concat_df["text"].apply(lambda r: len(r.split(turn_token)))

In [258]:
# Add column with utterance counts
concat_df["count"] = utterance_counts

In [259]:
# Get the most common utterance count
concat_df["count"].value_counts()

4    13738
5     3113
6      441
8      238
7      199
3       39
1       13
2       12
Name: count, dtype: int64

In [260]:
# Get utterance count that's most common
num_turns = concat_df["count"].value_counts().idxmax()
print(num_turns)

4


In [261]:
labels = sorted(concat_df["context"].unique())
print(len(labels))
labels

32


['afraid',
 'angry',
 'annoyed',
 'anticipating',
 'anxious',
 'apprehensive',
 'ashamed',
 'caring',
 'confident',
 'content',
 'devastated',
 'disappointed',
 'disgusted',
 'embarrassed',
 'excited',
 'faithful',
 'furious',
 'grateful',
 'guilty',
 'hopeful',
 'impressed',
 'jealous',
 'joyful',
 'lonely',
 'nostalgic',
 'prepared',
 'proud',
 'sad',
 'sentimental',
 'surprised',
 'terrified',
 'trusting']

In [262]:
# Explode text into individual rows
preprocessed_df = concat_df["text"].apply(lambda r: [f"<s{i % 2}>{s}" for i, s in enumerate(r.split(turn_token))]).explode().reset_index()
preprocessed_df

,index,text
0,0,<s0>sentimental I remember going to see the fi...
1,0,<s1>sentimental Was this a friend you were in ...
2,0,<s0>sentimental This was a best friend. I miss...
3,0,<s1>sentimental Where has she gone?
4,0,<s0>sentimental We no longer talk.
...,...,...
76609,17791,<s1>ashamed i understand. maybe next time you ...
76610,17792,<s0>embarrassed Once when I was in preschool_c...
76611,17792,<s1>embarrassed haha yeah that would be very b...
76612,17792,<s0>embarrassed It was_comma_ I almost cried c...


In [263]:
preprocessed_df.iloc[112]

index                                                   26
text     <s1>joyful Congratulations?  How much did you ...
Name: 112, dtype: object

In [264]:
df.iloc[112]

conv_id                                             hit:15_conv:31
utterance_idx                                                    1
context                                                     guilty
prompt           After learning my husband of only 4 weeks was ...
speaker_idx                                                     22
utterance        I was only married for 4 weeks when I learned ...
selfeval                                               5|5|5_5|5|5
tags                                                           NaN
tokenized_len                                                   47
text             guilty I was only married for 4 weeks when I l...
Name: 112, dtype: object

In [265]:
# get speaker
def get_speaker(text):
    results = re.findall(r"^<s\d>", text)
    return results
    
    
def clean_text(text):
    text = re.sub(r"^<s\d>", "", text)
    return re.sub(r"{turn_token}$", "", text)

In [266]:
get_speaker(preprocessed_df["text"].iloc[0])

['<s0>']

In [267]:
preprocessed_df["speaker"] = preprocessed_df["text"].apply(lambda t: get_speaker(t)[0])

In [268]:
preprocessed_df

,index,text,speaker
0,0,<s0>sentimental I remember going to see the fi...,<s0>
1,0,<s1>sentimental Was this a friend you were in ...,<s1>
2,0,<s0>sentimental This was a best friend. I miss...,<s0>
3,0,<s1>sentimental Where has she gone?,<s1>
4,0,<s0>sentimental We no longer talk.,<s0>
...,...,...,...
76609,17791,<s1>ashamed i understand. maybe next time you ...,<s1>
76610,17792,<s0>embarrassed Once when I was in preschool_c...,<s0>
76611,17792,<s1>embarrassed haha yeah that would be very b...,<s1>
76612,17792,<s0>embarrassed It was_comma_ I almost cried c...,<s0>


In [269]:
preprocessed_df["text"] = preprocessed_df["text"].apply(lambda t: clean_text(t)) ## NOTE: we're not replacing commas cuz it messes up the CSV

In [270]:
preprocessed_df.iloc[0].text

'sentimental I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.'

In [271]:
preprocessed_df

,index,text,speaker
0,0,sentimental I remember going to see the firewo...,<s0>
1,0,sentimental Was this a friend you were in love...,<s1>
2,0,sentimental This was a best friend. I miss her.,<s0>
3,0,sentimental Where has she gone?,<s1>
4,0,sentimental We no longer talk.,<s0>
...,...,...,...
76609,17791,ashamed i understand. maybe next time you shou...,<s1>
76610,17792,embarrassed Once when I was in preschool_comma...,<s0>
76611,17792,embarrassed haha yeah that would be very bad !,<s1>
76612,17792,embarrassed It was_comma_ I almost cried cause...,<s0>


In [272]:
preprocessed_df.drop(["index"], axis=1, inplace=True)

In [273]:
preprocessed_df.iloc[112]

text       joyful Congratulations?  How much did you win?
speaker                                              <s1>
Name: 112, dtype: object

In [275]:
preprocessed_df.to_csv(f"../../data/empathetic_dialogue_processed_{dataset_type}--cleaned128.csv")

### Prepare data

In [276]:
from src.utils import prepare_data

In [277]:
preprocessed_df

,text,speaker
0,sentimental I remember going to see the firewo...,<s0>
1,sentimental Was this a friend you were in love...,<s1>
2,sentimental This was a best friend. I miss her.,<s0>
3,sentimental Where has she gone?,<s1>
4,sentimental We no longer talk.,<s0>
...,...,...
76609,ashamed i understand. maybe next time you shou...,<s1>
76610,embarrassed Once when I was in preschool_comma...,<s0>
76611,embarrassed haha yeah that would be very bad !,<s1>
76612,embarrassed It was_comma_ I almost cried cause...,<s0>


In [278]:
preprocessed_df[preprocessed_df["speaker"] == "<s1>"]

,text,speaker
1,sentimental Was this a friend you were in love...,<s1>
3,sentimental Where has she gone?,<s1>
5,sentimental Oh was this something that happene...,<s1>
7,surprised Do you like toads?,<s1>
9,surprised That was nice of your girlfriend_com...,<s1>
...,...,...
76605,devastated He is a legend in the making,<s1>
76607,ashamed oh no :( maybe you should confess and ...,<s1>
76609,ashamed i understand. maybe next time you shou...,<s1>
76611,embarrassed haha yeah that would be very bad !,<s1>


In [209]:
#preprocessed_df["tokenized"] = preprocessed_df["text"].apply(lambda x: tokenizer.encode(clean_text(x)) + [tokenizer.eos_token_id])

In [210]:
#preprocessed_df["num_tokens"] = preprocessed_df["tokenized"].apply(lambda x: len(x))

In [212]:
# preprocessed_df["text"] = preprocessed_df["text"].apply(lambda t: t.replace("_comma_", ","))
# preprocessed_df[preprocessed_df["speaker"] == "<s1>"].sort_values(by="num_tokens", ascending=False)

,index,text,speaker,tokenized,num_tokens
15174,3416,jealous Yea_comma_ recently I have been trying...,<s1>,"[73, 15746, 11609, 64, 62, 785, 2611, 62, 2904...",146
54885,12688,lonely Healing is never easy. It always has th...,<s1>,"[75, 505, 306, 22508, 318, 1239, 2562, 13, 632...",118
37412,8576,lonely Ah_comma_ your situation sounds complic...,<s1>,"[75, 505, 306, 7900, 62, 785, 2611, 62, 534, 3...",117
37386,8570,caring I'm really sorry to hear that. What's w...,<s1>,"[66, 1723, 314, 1101, 1107, 7926, 284, 3285, 3...",115
36731,8417,jealous Do you feel negative about it_comma_ o...,<s1>,"[73, 15746, 2141, 345, 1254, 4633, 546, 340, 6...",112
...,...,...,...,...,...
66712,15444,caring thanks,<s1>,"[66, 1723, 5176, 50256]",4
66702,15441,confident ok,<s1>,"[10414, 738, 12876, 50256]",4
43705,10065,content What,<s1>,"[11299, 1867, 50256]",3
38686,8877,content nice,<s1>,"[11299, 3621, 50256]",3


In [216]:
#preprocessed_df.sort_values(by="num_tokens", ascending=False)

,index,text,speaker,tokenized,num_tokens
64092,14838,lonely there was a time when I lived in anothe...,<s0>,"[75, 505, 306, 612, 373, 257, 640, 618, 314, 5...",902
76546,17778,joyful I was getting ready for work when my bo...,<s0>,"[2633, 913, 314, 373, 1972, 3492, 329, 670, 61...",645
54936,12701,ashamed ...,<s0>,"[1077, 2434, 220, 220, 220, 220, 220, 220, 220...",618
6172,1381,caring I had a good friend who just lost their...,<s0>,"[66, 1723, 314, 550, 257, 922, 1545, 508, 655,...",540
16414,3699,impressed I was impressed with my friend when ...,<s0>,"[11011, 2790, 314, 373, 12617, 351, 616, 1545,...",457
...,...,...,...,...,...
51797,11978,content maybe,<s0>,"[11299, 3863, 50256]",3
39929,9169,content 1600,<s0>,"[11299, 26143, 50256]",3
7256,1614,content 1800,<s0>,"[11299, 21431, 50256]",3
43705,10065,content What,<s1>,"[11299, 1867, 50256]",3


In [222]:
#preprocessed_df[preprocessed_df["num_tokens"] <= 512].shape

(76623, 5)

In [221]:
# Only include reponse
#preprocessed_df = preprocessed_df[preprocessed_df["num_tokens"] <= 512]

,index,text,speaker,tokenized,num_tokens
0,0,sentimental I remember going to see the firewo...,<s0>,"[34086, 9134, 314, 3505, 1016, 284, 766, 262, ...",48
1,0,sentimental Was this a friend you were in love...,<s1>,"[34086, 9134, 8920, 428, 257, 1545, 345, 547, ...",22
2,0,sentimental This was a best friend. I miss her.,<s0>,"[34086, 9134, 770, 373, 257, 1266, 1545, 13, 3...",13
3,0,sentimental Where has she gone?,<s1>,"[34086, 9134, 6350, 468, 673, 3750, 30, 50256]",8
4,0,sentimental We no longer talk.,<s0>,"[34086, 9134, 775, 645, 2392, 1561, 13, 50256]",8
...,...,...,...,...,...
76622,17796,ashamed i understand. maybe next time you shou...,<s1>,"[1077, 2434, 1312, 1833, 13, 3863, 1306, 640, ...",14
76623,17797,embarrassed Once when I was in preschool_comma...,<s0>,"[24419, 9187, 276, 4874, 618, 314, 373, 287, 3...",34
76624,17797,embarrassed haha yeah that would be very bad !,<s1>,"[24419, 9187, 276, 42254, 10194, 326, 561, 307...",12
76625,17797,embarrassed It was_comma_ I almost cried cause...,<s0>,"[24419, 9187, 276, 632, 373, 62, 785, 2611, 62...",19


In [279]:
prepared_df = prepare_data(data=preprocessed_df, filter_by="speaker==<s1>", text_key="text", num_history=7, test_size=None)

In [281]:
prepared_df.sample()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
34417,disappointed Oh wow. Was it a bad car wreck?,disappointed I broke it in a car wreck!,disappointed Oh no my dear_comma_ what happene...,disappointed After all of this training_comma_...,lonely ah that is also there. You will make ne...,lonely Yes_comma_ that's true. I haven't lived...,lonely oh why not!! quality is better than qua...,lonely I just realized that I don't have many ...


In [282]:
prepared_df.to_csv(f"ed_train__{dataset_type}-cleaned128.csv")

In [109]:
from src.dataset import *

In [283]:
prepared_df

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,surprised Do you like toads?,surprised My girlfriend got me a pet toad today!,sentimental Oh was this something that happene...,sentimental We no longer talk.,sentimental Where has she gone?,sentimental This was a best friend. I miss her.,sentimental Was this a friend you were in love...,sentimental I remember going to see the firewo...
1,surprised That was nice of your girlfriend_com...,surprised I do! I was so happy when I opened t...,surprised Do you like toads?,surprised My girlfriend got me a pet toad today!,sentimental Oh was this something that happene...,sentimental We no longer talk.,sentimental Where has she gone?,sentimental This was a best friend. I miss her.
2,impressed That's nice. What color did you pain...,impressed I really like the new paint job on m...,surprised That was nice of your girlfriend_com...,surprised I do! I was so happy when I opened t...,surprised Do you like toads?,surprised My girlfriend got me a pet toad today!,sentimental Oh was this something that happene...,sentimental We no longer talk.
3,impressed Those are swell colors! I am sure th...,impressed I went with blue and yellow.,impressed That's nice. What color did you pain...,impressed I really like the new paint job on m...,surprised That was nice of your girlfriend_com...,surprised I do! I was so happy when I opened t...,surprised Do you like toads?,surprised My girlfriend got me a pet toad today!
4,lonely That's disappointing. I'm glad that you...,lonely I went to the skating rink all by mysel...,impressed Those are swell colors! I am sure th...,impressed I went with blue and yellow.,impressed That's nice. What color did you pain...,impressed I really like the new paint job on m...,surprised That was nice of your girlfriend_com...,surprised I do! I was so happy when I opened t...
...,...,...,...,...,...,...,...,...
36617,devastated He is a legend in the making,devastated Yea_comma_ I didnt think he had it ...,devastated I too was surprised. To leave the ...,devastated I was shocked when Lebron left the ...,confident Thank you_comma_ I should hear somet...,confident Good luck!,confident I am hoping to go to Mitchell Hamlin...,confident Ah thats a good score. When i took ...
36618,ashamed oh no :( maybe you should confess and ...,ashamed I cheated on a test. I am very ashamed.,devastated He is a legend in the making,devastated Yea_comma_ I didnt think he had it ...,devastated I too was surprised. To leave the ...,devastated I was shocked when Lebron left the ...,confident Thank you_comma_ I should hear somet...,confident Good luck!
36619,ashamed i understand. maybe next time you shou...,ashamed I would but I will get in trouble.,ashamed oh no :( maybe you should confess and ...,ashamed I cheated on a test. I am very ashamed.,devastated He is a legend in the making,devastated Yea_comma_ I didnt think he had it ...,devastated I too was surprised. To leave the ...,devastated I was shocked when Lebron left the ...
36620,embarrassed haha yeah that would be very bad !,embarrassed Once when I was in preschool_comma...,ashamed i understand. maybe next time you shou...,ashamed I would but I will get in trouble.,ashamed oh no :( maybe you should confess and ...,ashamed I cheated on a test. I am very ashamed.,devastated He is a legend in the making,devastated Yea_comma_ I didnt think he had it ...


In [113]:
ed_train = pd.read_csv("ed_train.csv")
ed_train__train = pd.read_csv("ed_train__train.csv")
ed_train__test = pd.read_csv("ed_train__test.csv")

In [114]:
ed_train__test.head(10)

,Unnamed: 0,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,0,"faithful My girlfriend, How about you? do you...","faithful I always do the same, it's not had w...",trusting Maybe A move out of my state would be...,trusting I have problems with the city and the...,"surprised It was, and he does that type of th...","surprised I got something nice the other day, ...",guilty I don't know I was new to driving and h...,guilty No I wasn't hit. It turned out they wer...
1,1,trusting Surprisingly it's been a painless pro...,"trusting I close on my new house Monday, I di...",faithful Wow. Looks like you are doing a good ...,"faithful My girlfriend, How about you? do you...","faithful I always do the same, it's not had w...",trusting Maybe A move out of my state would be...,trusting I have problems with the city and the...,"surprised It was, and he does that type of th..."
2,2,trusting I really thought I could count on he...,trusting I gave my friend some money that she ...,trusting Surprisingly it's been a painless pro...,"trusting I close on my new house Monday, I di...",faithful Wow. Looks like you are doing a good ...,"faithful My girlfriend, How about you? do you...","faithful I always do the same, it's not had w...",trusting Maybe A move out of my state would be...
3,3,ashamed I stole my mother's card details to pa...,"ashamed I actually did this, i'm terrible.",trusting I really thought I could count on he...,trusting I gave my friend some money that she ...,trusting Surprisingly it's been a painless pro...,"trusting I close on my new house Monday, I di...",faithful Wow. Looks like you are doing a good ...,"faithful My girlfriend, How about you? do you..."
4,4,apprehensive I'm really hesitant to get on. It...,apprehensive We are going to Six Flags this we...,ashamed I stole my mother's card details to pa...,"ashamed I actually did this, i'm terrible.",trusting I really thought I could count on he...,trusting I gave my friend some money that she ...,trusting Surprisingly it's been a painless pro...,"trusting I close on my new house Monday, I di..."
5,5,devastated It is a horrible experience and i h...,devastated When i lost my dog of 12 years i wa...,apprehensive I'm really hesitant to get on. It...,apprehensive We are going to Six Flags this we...,ashamed I stole my mother's card details to pa...,"ashamed I actually did this, i'm terrible.",trusting I really thought I could count on he...,trusting I gave my friend some money that she ...
6,6,caring Money mostly. I just wish i could have ...,caring I think that in the past i have spent t...,devastated It is a horrible experience and i h...,devastated When i lost my dog of 12 years i wa...,apprehensive I'm really hesitant to get on. It...,apprehensive We are going to Six Flags this we...,ashamed I stole my mother's card details to pa...,"ashamed I actually did this, i'm terrible."
7,7,sentimental It really is. All the neighbors we...,sentimental I love thinking back to when I was...,caring Money mostly. I just wish i could have ...,caring I think that in the past i have spent t...,devastated It is a horrible experience and i h...,devastated When i lost my dog of 12 years i wa...,apprehensive I'm really hesitant to get on. It...,apprehensive We are going to Six Flags this we...
8,8,content What type of content do you like watch...,content Do you watch any YouTube at all?,sentimental It really is. All the neighbors we...,sentimental I love thinking back to when I was...,caring Money mostly. I just wish i could have ...,caring I think that in the past i have spent t...,devastated It is a horrible experience and i h...,devastated When i lost my dog of 12 years i wa...
9,9,ashamed The worst part was that there was a ca...,ashamed The other night I wanted cake when I w...,content What type of content do you like watch...,content Do you watch any YouTube at all?,sentimental It really is. All the neighbors we...,sentimental I love thinking back to when I was..

## Counsel Chat Data

In [12]:
counselchat_data = pd.read_csv("../../data/counsel_chat/counselchat-data.csv")

In [13]:
counselchat_data.shape

(1482, 9)

In [14]:
counselchat_data.head(5)

,questionID,questionTitle,questionText,questionUrl,topics,therapistName,therapistUrl,answerText,upvotes
0,5566fab2a64752d71ec3ca69,Escalating disagreements between mother and wife,My wife and mother are having tense disagreeme...,https://counselchat.com/questions/escalating-d...,Family Conflict,"Kristi King-Morgan, LMSW",https://counselchat.com/therapists/kristi-king...,<p>What you are describing is something psycho...,0
1,5566f94fa64752d71ec3ca64,I'm addicted to smoking. How can I stop?,"I'm planning to have baby, so I have to quit s...",https://counselchat.com/questions/i-m-addicted...,"Substance Abuse,Addiction",Rebecca Duellman,https://counselchat.com/therapists/rebecca-due...,<p>Hi. Good for you in planning ahead to do wh...,0
2,5567d26887a1cc0c3f3d8f46,Keeping secrets from my family,"I have secrets in my mind, and I don't know wh...",https://counselchat.com/questions/keeping-secr...,Family Conflict,Jeevna Bajaj,https://counselchat.com/therapists/jeevna-bajaj,<p>It sounds like keeping the secrets has beco...,0
3,556bed15c969ba5861709df5,The Underlying Causes of Being Possessive,I am extremely possessive in my relationships ...,https://counselchat.com/questions/the-underlyi...,"Behavioral Change,Social Relationships",Rebecca Duellman,https://counselchat.com/therapists/rebecca-due...,<p>Hi there. It's great you are able to realiz...,0
4,556ba115c969ba5861709de6,Can I control anxiety without medication?,I had a head injury a few years ago and my min...,https://counselchat.com/questions/can-i-contro...,Anxiety,Rebecca Duellman,https://counselchat.com/therapists/rebecca-due...,<p>You didn't say what or how many medications...,0


In [16]:
counselchat_data[["questionText", "answerText"]]

,questionText,answerText
0,My wife and mother are having tense disagreeme...,<p>What you are describing is something psycho...
1,"I'm planning to have baby, so I have to quit s...",<p>Hi. Good for you in planning ahead to do wh...
2,"I have secrets in my mind, and I don't know wh...",<p>It sounds like keeping the secrets has beco...
3,I am extremely possessive in my relationships ...,<p>Hi there. It's great you are able to realiz...
4,I had a head injury a few years ago and my min...,<p>You didn't say what or how many medications...
...,...,...
1477,My grandson's step-mother sends him to school ...,<p>Absolutely not!&nbsp;</p><p>It is never in ...
1478,My boyfriend is in recovery from drug addictio...,<p>I'm sorry you have tension between you and ...
1479,The birth mother attempted suicide several tim...,"<p>The true answer is, ""no one can really say ..."
1480,I think adult life is making him depressed and...,<p>How do you help yourself to believe you req...


In [24]:
answer_text = counselchat_data[["questionText", "answerText"]].iloc[0]["answerText"]
answer_text

"<p>What you are describing is something psychologists have termed &#34;triangulation&#34; which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been &#34;triangulated&#34; by your wife and mother.</p><p>This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, especially in relationships where there is a power differential. For example, I bet it's easier to complain to a coworker about your boss rather than go to the boss with your complaint.</p><p>I'm not saying triangulation is always a bad thing. Sometimes a third party mediator is needed to help solve problems between two people who disagree. That's what therapists do every day! Sometimes just getting someone else's perspective can help you 

In [25]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(answer_text, "html.parser")

In [27]:
soup.find_all("p")

[<p>What you are describing is something psychologists have termed "triangulation" which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been "triangulated" by your wife and mother.</p>,
 <p>This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, especially in relationships where there is a power differential. For example, I bet it's easier to complain to a coworker about your boss rather than go to the boss with your complaint.</p>,
 <p>I'm not saying triangulation is always a bad thing. Sometimes a third party mediator is needed to help solve problems between two people who disagree. That's what therapists do every day! Sometimes just getting someone else's perspective can help you see the is

In [17]:
import json

In [260]:
with open("../../data/counsel_chat/counsel_chat_250-tokens_full.json") as fp:
    cc_data = json.load(fp)

In [261]:
cc_data.keys()

dict_keys(['train', 'valid'])

In [262]:
print("training_examples: {num_train}\nvalid_examples: {num_val}".format(num_train=len(cc_data["train"]), num_val=len(cc_data["valid"])))

training_examples: 1839
valid_examples: 173


In [263]:
data["train"][:10]

[{'personality': [''],
  'utterances': [{'history': ["can i change my feeling of being worthless to everyone ? i ' m going through some things with my feelings and myself . i barely sleep and i do nothing but think about how i ' m worthless and how i shouldn ' t be here . i ' ve never tried or contemplated suicide . i ' ve always wanted to fix my issues , but i never get around to it . how can i change my feeling of being worthless to everyone ?"],
    'candidates': ['maybe lower your expectations for a bit',
     'if you are whole - heartedly committed to moving past the sexual and romantic parts of your relationship and just having a friendship than refraining from all the touching would be a good place to start',
     'very often , one person wants to deal with the conflict right away or shortly thereafter and the other person wants to wait',
     '" my best guess is that your boyfriend is triggered by some previous relationship , either romantic or in childhood',
     'can he do th

In [59]:
data["train"][100]["utterances"][0]["candidates"]

['marriage is a people growing process ; you mature into one another and to achieve fulfillment your continued development will be required',
 "is it possible what you ' re feeling is from being reminded of great distress you lived through when you were at a similar age as the kids around whom you feel this sudden and great paranoia and fear",
 'if it is possible , see if he would be willing to go to a counselor with you',
 'it is common with some types of seizures to have an altered state of reality during or shortly after the seizure , so that could also be related',
 'allow yourself some time to reflect on how you feel toward your bf',
 'i am wishing my very best to you',
 'since you are aware of your tendencies and how the interfere with your life , try to become aware of when you do these habits',
 'you know , it does sounds like you have imagined a motherly comfort figure for yourself',
 'at present , the american disability association ( ada ) only allows protection and guiltine

In [27]:
data["train"][100]["utterances"][0]["history"]

["my apartment manager won ' t let me keep an emotional support dog i have been diagnosed with general anxiety and depression by my family doctor . they wrote a prescription for me to have an emotional support dog , i have the paper work , and i gave it to my apartment manager . they said i can ' t keep the esd because i ' m not disabled . what do you suggest i do ?"]

In [61]:
# https://huggingface.co/j-hartmann/emotion-english-distilroberta-base
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
classifier("I love this!")

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

[[{'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'disgust', 'score': 0.0016119900392368436},
  {'label': 'fear', 'score': 0.0004138521908316761},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'surprise', 'score': 0.008528684265911579}]]

In [62]:
from datasets import load_dataset

In [63]:
dataset = load_dataset("empathetic_dialogues")

Using custom data configuration default


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Dataset empathetic_dialogues downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [64]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})

In [67]:
dataset["train"][0]

{'conv_id': 'hit:0_conv:1',
 'utterance_idx': 1,
 'context': 'sentimental',
 'prompt': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.',
 'speaker_idx': 1,
 'utterance': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.',
 'selfeval': '5|5|5_2|2|5',
 'tags': ''}

In [68]:
df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN
...,...,...,...,...,...,...,...,...
76663,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5,NaN
76664,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5,NaN
76665,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5,NaN
76666,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5,NaN


# Training Emotion Classifier

In [70]:
classification = df[["context", "utterance"]]

In [71]:
classification

,context,utterance
0,sentimental,I remember going to see the fireworks with my ...
1,sentimental,Was this a friend you were in love with_comma_...
2,sentimental,This was a best friend. I miss her.
3,sentimental,Where has she gone?
4,sentimental,We no longer talk.
...,...,...
76663,sentimental,Yeah reminds me of the good old days. I miss ...
76664,surprised,I woke up this morning to my wife telling me s...
76665,surprised,Oh hey that's awesome! That is awesome right?
76666,surprised,It is soooo awesome. We have been wanting a b...


In [73]:
classification.rename(columns={"context": "label", "utterance": "text"}, inplace=True)

/tmp/ipykernel_1637669/307513464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification.rename(columns={"context": "label", "utterance": "text"}, inplace=True)


In [90]:
classification["text"] = classification["text"].apply(lambda i: i.replace("_comma_", ","))

/tmp/ipykernel_1637669/3200956030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification["text"] = classification["text"].apply(lambda i: i.replace("_comma_", ","))


In [171]:
classification

,label,text
0,sentimental,I remember going to see the fireworks with my ...
1,sentimental,"Was this a friend you were in love with, or ju..."
2,sentimental,This was a best friend. I miss her.
3,sentimental,Where has she gone?
4,sentimental,We no longer talk.
...,...,...
76663,sentimental,Yeah reminds me of the good old days. I miss ...
76664,surprised,I woke up this morning to my wife telling me s...
76665,surprised,Oh hey that's awesome! That is awesome right?
76666,surprised,It is soooo awesome. We have been wanting a b...


In [173]:
labels_map

[('afraid', 0),
 ('angry', 1),
 ('annoyed', 2),
 ('anticipating', 3),
 ('anxious', 4),
 ('apprehensive', 5),
 ('ashamed', 6),
 ('caring', 7),
 ('confident', 8),
 ('content', 9),
 ('devastated', 10),
 ('disappointed', 11),
 ('disgusted', 12),
 ('embarrassed', 13),
 ('excited', 14),
 ('faithful', 15),
 ('furious', 16),
 ('grateful', 17),
 ('guilty', 18),
 ('hopeful', 19),
 ('impressed', 20),
 ('jealous', 21),
 ('joyful', 22),
 ('lonely', 23),
 ('nostalgic', 24),
 ('prepared', 25),
 ('proud', 26),
 ('sad', 27),
 ('sentimental', 28),
 ('surprised', 29),
 ('terrified', 30),
 ('trusting', 31)]

In [112]:
from sklearn.model_selection import train_test_split

In [166]:
labels = classification[["label"]].sort_values("label").drop_duplicates().reset_index(drop=True)

In [174]:
labels_map = dict(list(zip(labels.label, labels.index)))

In [175]:
labels_map

{'afraid': 0,
 'angry': 1,
 'annoyed': 2,
 'anticipating': 3,
 'anxious': 4,
 'apprehensive': 5,
 'ashamed': 6,
 'caring': 7,
 'confident': 8,
 'content': 9,
 'devastated': 10,
 'disappointed': 11,
 'disgusted': 12,
 'embarrassed': 13,
 'excited': 14,
 'faithful': 15,
 'furious': 16,
 'grateful': 17,
 'guilty': 18,
 'hopeful': 19,
 'impressed': 20,
 'jealous': 21,
 'joyful': 22,
 'lonely': 23,
 'nostalgic': 24,
 'prepared': 25,
 'proud': 26,
 'sad': 27,
 'sentimental': 28,
 'surprised': 29,
 'terrified': 30,
 'trusting': 31}

In [181]:
train_df, val_df = train_test_split(classification, test_size=0.2)
train_df["label"].replace(labels_map, inplace=True)
val_df["label"].replace(labels_map, inplace=True)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

In [182]:
train_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 61334
})

In [183]:
dataset = DatasetDict({"train": train_dataset, "test": val_dataset})

In [184]:
dataset["train"][0]

{'label': 19,
 'text': 'Wow, that was very generous and courageous for her! How are they now?'}

In [185]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=num_labels)

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_

In [186]:
from transformers import TrainingArguments

output_dir = "../../models/distilroberta-finetuned"

training_args = TrainingArguments(output_dir=output_dir)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [103]:
! ls ../../models

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
archives  hopperbot-medium


In [187]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [188]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [189]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=output_dir, evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [193]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_siz

  0%|          | 0/62 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

In [194]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [195]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 61334
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 23001
Trainer is attempting to log a value of "{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7', 8: 'LABEL_8', 9: 'LABEL_

Epoch,Training Loss,Validation Loss,Accuracy
1,2.520400,2.507685,0.284205
2,2.287800,2.438132,0.308791
3,1.955100,2.479506,0.319421


Saving model checkpoint to ../../models/distilroberta-finetuned/checkpoint-500
Configuration saved in ../../models/distilroberta-finetuned/checkpoint-500/config.json


Attempted to log scalar metric loss:
3.1307
Attempted to log scalar metric learning_rate:
4.891309073518543e-05
Attempted to log scalar metric epoch:
0.07


Model weights saved in ../../models/distilroberta-finetuned/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ../../models/distilroberta-finetuned/checkpoint-1000
Configuration saved in ../../models/distilroberta-finetuned/checkpoint-1000/config.json


...

Model weights saved in ../../models/distilroberta-finetuned/checkpoint-23000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15334
  Batch size = 8


Attempted to log scalar metric eval_loss:
2.4795055389404297
Attempted to log scalar metric eval_accuracy:
0.3194208947437068
Attempted to log scalar metric eval_runtime:
89.9825
Attempted to log scalar metric eval_samples_per_second:
170.411
Attempted to log scalar metric eval_steps_per_second:
21.304
Attempted to log scalar metric epoch:
3.0




Training completed. Do not forget to share your model on huggingface.co/models =)




Attempted to log scalar metric train_runtime:
3591.6709
Attempted to log scalar metric train_samples_per_second:
51.23
Attempted to log scalar metric train_steps_per_second:
6.404
Attempted to log scalar metric total_flos:
2.438730670060339e+16
Attempted to log scalar metric train_loss:
2.3298980236924174
Attempted to log scalar metric epoch:
3.0


TrainOutput(global_step=23001, training_loss=2.3298980236924174, metrics={'train_runtime': 3591.6709, 'train_samples_per_second': 51.23, 'train_steps_per_second': 6.404, 'total_flos': 2.438730670060339e+16, 'train_loss': 2.3298980236924174, 'epoch': 3.0})

In [199]:
trainer.save_model(output_dir)

Saving model checkpoint to ../../models/distilroberta-finetuned
Configuration saved in ../../models/distilroberta-finetuned/config.json
Model weights saved in ../../models/distilroberta-finetuned/pytorch_model.bin


In [204]:
tokenizer.save_pretrained(output_dir)
tokenizer.save_vocabulary(output_dir)

tokenizer config file saved in ../../models/distilroberta-finetuned/tokenizer_config.json
Special tokens file saved in ../../models/distilroberta-finetuned/special_tokens_map.json


('../../models/distilroberta-finetuned/vocab.json',
 '../../models/distilroberta-finetuned/merges.txt')

In [211]:
id2label = dict((v,k) for k,v in labels_map.items())

In [212]:
from transformers import pipeline
classifier = pipeline("text-classification", model=output_dir, return_all_scores=True)

loading configuration file ../../models/distilroberta-finetuned/config.json
Model config RobertaConfig {
  "_name_or_path": "../../models/distilroberta-finetuned",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL

In [215]:
classifier.model.config.id2label = id2label
classifier.model.config.label2id = labels_map

In [216]:
classifier.model.config

RobertaConfig {
  "_name_or_path": "../../models/distilroberta-finetuned",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "afraid",
    "1": "angry",
    "2": "annoyed",
    "3": "anticipating",
    "4": "anxious",
    "5": "apprehensive",
    "6": "ashamed",
    "7": "caring",
    "8": "confident",
    "9": "content",
    "10": "devastated",
    "11": "disappointed",
    "12": "disgusted",
    "13": "embarrassed",
    "14": "excited",
    "15": "faithful",
    "16": "furious",
    "17": "grateful",
    "18": "guilty",
    "19": "hopeful",
    "20": "impressed",
    "21": "jealous",
    "22": "joyful",
    "23": "lonely",
    "24": "nostalgic",
    "25": "prepared",
    "26": "proud",
    "27": "sad",
    "28": "sentimental",
    "29": "surprised",
    "30": "

In [220]:
classifier("I'm still grieving the passing of my father. Sometimes I think, I'll never get over it")

[[{'label': 'afraid', 'score': 0.001271251356229186},
  {'label': 'angry', 'score': 0.002481884090229869},
  {'label': 'annoyed', 'score': 0.0008107623434625566},
  {'label': 'anticipating', 'score': 0.00017449459119234234},
  {'label': 'anxious', 'score': 0.0008662308100610971},
  {'label': 'apprehensive', 'score': 0.00039876039954833686},
  {'label': 'ashamed', 'score': 0.00147321040276438},
  {'label': 'caring', 'score': 0.0030949898064136505},
  {'label': 'confident', 'score': 0.00012000662536593154},
  {'label': 'content', 'score': 0.00021913884847890586},
  {'label': 'devastated', 'score': 0.40299463272094727},
  {'label': 'disappointed', 'score': 0.005933469161391258},
  {'label': 'disgusted', 'score': 0.0012640400091186166},
  {'label': 'embarrassed', 'score': 0.0018726286944001913},
  {'label': 'excited', 'score': 0.00021567723888438195},
  {'label': 'faithful', 'score': 0.0010265909368172288},
  {'label': 'furious', 'score': 0.0019771941006183624},
  {'label': 'grateful', 'sc

In [221]:
classifier("Life is meaningless.")

[[{'label': 'afraid', 'score': 0.006920903921127319},
  {'label': 'angry', 'score': 0.016934223473072052},
  {'label': 'annoyed', 'score': 0.00985314417630434},
  {'label': 'anticipating', 'score': 0.001343411160632968},
  {'label': 'anxious', 'score': 0.004305040463805199},
  {'label': 'apprehensive', 'score': 0.004895302467048168},
  {'label': 'ashamed', 'score': 0.004045652225613594},
  {'label': 'caring', 'score': 0.004559752531349659},
  {'label': 'confident', 'score': 0.0012360070832073689},
  {'label': 'content', 'score': 0.008918428793549538},
  {'label': 'devastated', 'score': 0.11213884502649307},
  {'label': 'disappointed', 'score': 0.030838578939437866},
  {'label': 'disgusted', 'score': 0.00690632127225399},
  {'label': 'embarrassed', 'score': 0.005127838812768459},
  {'label': 'excited', 'score': 0.0015857134712859988},
  {'label': 'faithful', 'score': 0.013053747825324535},
  {'label': 'furious', 'score': 0.01415031123906374},
  {'label': 'grateful', 'score': 0.004609598

In [226]:
classifier(["Congratulations on your wedding!", "Who am I kidding? I'm never gonna get that.", "Going to that festival was my favorite childhood activity."])

[[{'label': 'afraid', 'score': 0.0009797802194952965},
  {'label': 'angry', 'score': 0.0005866821738891304},
  {'label': 'annoyed', 'score': 0.0006858629640191793},
  {'label': 'anticipating', 'score': 0.0569567009806633},
  {'label': 'anxious', 'score': 0.018783459439873695},
  {'label': 'apprehensive', 'score': 0.002131709596142173},
  {'label': 'ashamed', 'score': 0.001870174310170114},
  {'label': 'caring', 'score': 0.009022172540426254},
  {'label': 'confident', 'score': 0.036349739879369736},
  {'label': 'content', 'score': 0.06657946854829788},
  {'label': 'devastated', 'score': 0.003845076309517026},
  {'label': 'disappointed', 'score': 0.0017840979853644967},
  {'label': 'disgusted', 'score': 0.0005492960917763412},
  {'label': 'embarrassed', 'score': 0.0008198667201213539},
  {'label': 'excited', 'score': 0.1879899799823761},
  {'label': 'faithful', 'score': 0.03524446487426758},
  {'label': 'furious', 'score': 0.00036482090945355594},
  {'label': 'grateful', 'score': 0.01934

In [228]:
results = classifier("I'm still grieving the passing of my father. Sometimes I think, I'll never get over it")

In [230]:
results[0]

[{'label': 'afraid', 'score': 0.001271251356229186},
 {'label': 'angry', 'score': 0.002481884090229869},
 {'label': 'annoyed', 'score': 0.0008107623434625566},
 {'label': 'anticipating', 'score': 0.00017449459119234234},
 {'label': 'anxious', 'score': 0.0008662308100610971},
 {'label': 'apprehensive', 'score': 0.00039876039954833686},
 {'label': 'ashamed', 'score': 0.00147321040276438},
 {'label': 'caring', 'score': 0.0030949898064136505},
 {'label': 'confident', 'score': 0.00012000662536593154},
 {'label': 'content', 'score': 0.00021913884847890586},
 {'label': 'devastated', 'score': 0.40299463272094727},
 {'label': 'disappointed', 'score': 0.005933469161391258},
 {'label': 'disgusted', 'score': 0.0012640400091186166},
 {'label': 'embarrassed', 'score': 0.0018726286944001913},
 {'label': 'excited', 'score': 0.00021567723888438195},
 {'label': 'faithful', 'score': 0.0010265909368172288},
 {'label': 'furious', 'score': 0.0019771941006183624},
 {'label': 'grateful', 'score': 0.0006073678

In [233]:
max_item = max(results[0], key=lambda x:x["score"])

In [234]:
max_item

{'label': 'sad', 'score': 0.5276297926902771}

In [567]:
result = results[0]

In [576]:
result.sort(key=lambda item: item.get("score"), reverse=True)

In [578]:
result[:3]

[{'label': 'sad', 'score': 0.5276297926902771},
 {'label': 'devastated', 'score': 0.40299463272094727},
 {'label': 'sentimental', 'score': 0.024283558130264282}]

In [235]:
print(output_dir)

../../models/distilroberta-finetuned


In [245]:
from transformers import pipeline

label2id = {'afraid': 0, 'angry': 1, 'annoyed': 2, 'anticipating': 3, 'anxious': 4, 'apprehensive': 5, 'ashamed': 6, 'caring': 7, 'confident': 8, 'content': 9, 'devastated': 10, 'disappointed': 11, 'disgusted': 12, 'embarrassed': 13, 'excited': 14, 'faithful': 15, 'furious': 16, 'grateful': 17, 'guilty': 18, 'hopeful': 19, 'impressed': 20, 'jealous': 21, 'joyful': 22, 'lonely': 23, 'nostalgic': 24, 'prepared': 25, 'proud': 26, 'sad': 27, 'sentimental': 28, 'surprised': 29, 'terrified': 30, 'trusting': 31}
id2label = dict((v,k) for k,v in label2id.items())

classifier = pipeline("text-classification", model=output_dir, return_all_scores=True)
classifier.model.config.id2label = id2label
classifier.model.config.label2id = label2id

def classify_emotion(text):
    results = classifier(text)
    max_score = max(results[0], key=lambda x:x["score"])
    return max_score["label"]


loading configuration file ../../models/distilroberta-finetuned/config.json
Model config RobertaConfig {
  "_name_or_path": "../../models/distilroberta-finetuned",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL

In [250]:
classify_emotion("I wonder whether I'll decide one day that I'm tired of people and simply run off to the woods to live a life of solitude.")

'lonely'

In [251]:
text = """
These are the thoughts that come to mind when I reflect on my performance: I am incompetent. I am worthless. I am not worth saving. I should be dead. I should have died years ago. I should  make it easier for everyone and just die now. 
"""

classify_emotion(text)

'ashamed'

In [238]:
print(labels_map)

{'afraid': 0, 'angry': 1, 'annoyed': 2, 'anticipating': 3, 'anxious': 4, 'apprehensive': 5, 'ashamed': 6, 'caring': 7, 'confident': 8, 'content': 9, 'devastated': 10, 'disappointed': 11, 'disgusted': 12, 'embarrassed': 13, 'excited': 14, 'faithful': 15, 'furious': 16, 'grateful': 17, 'guilty': 18, 'hopeful': 19, 'impressed': 20, 'jealous': 21, 'joyful': 22, 'lonely': 23, 'nostalgic': 24, 'prepared': 25, 'proud': 26, 'sad': 27, 'sentimental': 28, 'surprised': 29, 'terrified': 30, 'trusting': 31}


In [239]:
print(id2label)


{0: 'afraid', 1: 'angry', 2: 'annoyed', 3: 'anticipating', 4: 'anxious', 5: 'apprehensive', 6: 'ashamed', 7: 'caring', 8: 'confident', 9: 'content', 10: 'devastated', 11: 'disappointed', 12: 'disgusted', 13: 'embarrassed', 14: 'excited', 15: 'faithful', 16: 'furious', 17: 'grateful', 18: 'guilty', 19: 'hopeful', 20: 'impressed', 21: 'jealous', 22: 'joyful', 23: 'lonely', 24: 'nostalgic', 25: 'prepared', 26: 'proud', 27: 'sad', 28: 'sentimental', 29: 'surprised', 30: 'terrified', 31: 'trusting'}


In [563]:
help(json.dump)

Help on function dump in module json:

dump(obj, fp, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)
    Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
    ``.write()``-supporting file-like object).
    
    If ``skipkeys`` is true then ``dict`` keys that are not basic types
    (``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
    instead of raising a ``TypeError``.
    
    If ``ensure_ascii`` is false, then the strings written to ``fp`` can
    contain non-ASCII characters if they appear in strings contained in
    ``obj``. Otherwise, all such characters are escaped in JSON strings.
    
    If ``check_circular`` is false, then the circular reference check
    for container types will be skipped and a circular reference will
    result in an ``RecursionError`` (or worse).
    
    If ``allow_nan`` is false, then it will be a ``ValueError`` to
    seriali

In [565]:
with open("../../models/distilroberta-finetuned__exported/label2id.json", "w+") as f:
    json.dump(label2id, f)
    
with open("../../models/distilroberta-finetuned__exported/id2label.json", "w+") as f:
    json.dump(id2label, f)
    

In [252]:
from src.utils import export_model, make_tarfile

In [256]:
help(export_model)

Help on function export_model in module src.utils:

export_model(model_path, output_path)



In [259]:
export_model(output_dir)

loading configuration file ../../models/distilroberta-finetuned/config.json
Model config RobertaConfig {
  "_name_or_path": "../../models/distilroberta-finetuned",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL

Saved to ../../models/distilroberta-finetuned__exported


In [269]:
classify_emotion("The exhaustion of the soul can sometimes manifest itself as the exhaustion of the body.")

'anxious'

In [270]:
classify_emotion("I arrive home everyday feeling all my energy spent. There are days when I would much rather retire to my bed than eat dinner or read a book.")

'content'

In [271]:
classify_emotion("I have lost my sense of purpose. That may be why these dark feelings have remained. ")

'lonely'

In [272]:
classify_emotion("I can’t be honest with anyone, because nobody seems to validate what I’m going through. ")

'faithful'

In [273]:
classify_emotion("I’m absolutely miserable. I’m always made the villain here, rather than the one trying to get through her life. ")

'sad'

In [274]:
classify_emotion("Does anyone realize I’m actively suicidal? Probably not. I’ve not really indicated that I’ve been feeling suicidal.")

'apprehensive'

In [275]:
classify_emotion("Will I lie to try to get out of there without being put in the hospital?")

'guilty'

In [580]:
with open("test.json", "w+") as f:
    json.dump(id2label, f)

In [582]:
id2label[0]

'afraid'

# Test the Data

In [903]:
import pandas as pd

In [909]:
test_df = pd.read_csv("../../data/empatheticdialogues/valid.csv", on_bad_lines="skip")

In [911]:
test_df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:3_conv:6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,NaN
1,hit:3_conv:6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,NaN
2,hit:3_conv:6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,NaN
3,hit:4_conv:9,1,surprised,I was walking through my hallway a few week ag...,8,A few weeks ago_comma_ I was walking through m...,5|5|5_3|5|5,NaN
4,hit:4_conv:9,3,surprised,I was walking through my hallway a few week ag...,8,I may have let out a scream that will have him...,5|5|5_3|5|5,NaN
...,...,...,...,...,...,...,...,...
6313,hit:12350_conv:24701,5,prepared,When I went into labor with my son_comma_ it w...,791,In the end_comma_ yes--growing too big is actu...,5|5|5_5|5|5,NaN
6314,hit:12361_conv:24722,1,prepared,One time I studied all night for my final exam!,547,One time I studied all night for my final exam!,4|4|5_4|4|3,NaN
6315,hit:12361_conv:24722,3,prepared,One time I studied all night for my final exam!,547,It was for Organic Chemistry,4|4|5_4|4|3,NaN
6316,hit:12392_conv:24785,1,furious,One of my coworkers has been arguing with his ...,791,One of my coworkers has been arguing with his ...,4|5|5_5|5|5,NaN


In [915]:
df[["utterance"]].iloc[:]

,utterance
0,I remember going to see the fireworks with my ...
1,Was this a friend you were in love with_comma_...
2,This was a best friend. I miss her.
3,Where has she gone?
4,We no longer talk.
5,Oh was this something that happened because of...
6,it feels like hitting to blank wall when i se...
7,Oh ya? I don't really see how
8,dont you feel so.. its a wonder
9,I do actually hit blank walls a lot of times b...


In [917]:
len("I just don't feel confident about anything".split(" "))

7